# Assignment 2 - Data Cleaning

**Group number**: A-5 <br>
**Name student**: Zanou Rih & Sanaa El Marbouh<br>
**Student Number**: 300178068 & 300267783 <br>

##1. Introduction


##2. Dataset Description - Cafe Sales Dataset

&#128279; **Link**: https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training<br>
&#128100; **Author**: Ahmed Mohamed <br>

**Size**: 8 Columns, 10000 rows <br>
**Description**:  <br>

**About dataset**: <br>
The Dirty Cafe Sales dataset contains 10,000 rows of synthetic data representing sales transactions in a cafe. This dataset is intentionally "dirty," with missing values, inconsistent data, and errors introduced to provide a realistic scenario for data cleaning and exploratory data analysis (EDA). It can be used to practice cleaning techniques, data wrangling, and feature engineering.

**Key Features**: <br>
Transaction ID:	A unique identifier for each transaction. Always present and unique

Item:	The name of the item purchased. May contain missing or invalid values (e.g., "ERROR")

Quantity:	The quantity of the item purchased. May contain missing or invalid values

Price Per Unit:	The price of a single unit of the item. May contain missing or invalid value

Total Spent:	The total amount spent on the transaction. Calculated as Quantity * Price Per Unit

Payment Method:	The method of payment used. May contain missing or invalid values (e.g., None, "UNKNOWN")

Location	The location where the transaction occurred. May contain missing or invalid values

Transaction Date:	The date of the transaction. May contain missing or incorrect values.


In [5]:
# IMPORT STATEMENTS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


In [6]:
# raw file URL from github
git_url_ds = "https://raw.githubusercontent.com/zanoudev/csi4142-ds/refs/heads/master/dirty_cafe_sales.csv"

#load DS
ds = pd.read_csv(git_url_ds)

# display the first few rows
ds.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


##3. Errors
###3.1 Data Type Errors

In [7]:
# PARAMETERS

data_type_rules = {
    "Transaction ID": "string",
    "Item": "string",
    "Quantity": "int",
    "Price Per Unit": "float",
    "Total Spent": "float",
    "Payment Method": "string",
    "Location": "string",
    "Transaction Date": "datetime"
}

In [8]:
# CODE

data_type_errors = {}

# Check each column's data type
for col, expected_type in data_type_rules.items():
    if col in ds.columns:
        actual_type = ds[col].dtype
        invalid_values = []  # store all invalid values for frequency count

        # Check for datetime conversion
        if expected_type == "datetime":
            invalid_dates = ds[pd.to_datetime(ds[col], errors='coerce').isna()][col].tolist()
            invalid_values.extend(invalid_dates)

        # Check integer columns
        elif expected_type == "int":
            invalid_ints = ds[~ds[col].astype(str).str.match(r"^\d+$", na=False)][col].tolist()
            invalid_values.extend(invalid_ints)

        # Check float columns
        elif expected_type == "float":
            invalid_floats = ds[~ds[col].astype(str).str.match(r"^-?\d+(\.\d+)?$", na=False)][col].tolist()
            invalid_values.extend(invalid_floats)

        # Check string columns
        elif expected_type == "string":
            non_string_values = ds[ds[col].apply(lambda x: not isinstance(x, str))][col].tolist()
            invalid_values.extend(non_string_values)

        # Store errors only if there are any
        if invalid_values:
            error_counts = dict(Counter(invalid_values))  # Count occurrences
            total_errors = sum(error_counts.values())  # Total count
            data_type_errors[col] = {
                "Total Errors": total_errors,
                "Invalid Values": error_counts  # Show frequency of each invalid value
            }
    else:
        data_type_errors[col] = {
            "Total Errors": "Column Missing",
            "Invalid Values": {}
        }

# Results
if data_type_errors:
    print("Data Type Errors Found:")
    for col, details in data_type_errors.items():
        print(f"- {col}: {details['Total Errors']} errors")
        if details["Invalid Values"]:
            print("  Invalid Values:")
            for value, count in details["Invalid Values"].items():
                print(f"    - {repr(value)}: {count} times")
else:
    print("No data type errors found.")


Data Type Errors Found:
- Item: 333 errors
  Invalid Values:
    - nan: 333 times
- Quantity: 479 errors
  Invalid Values:
    - 'ERROR': 170 times
    - 'UNKNOWN': 171 times
    - nan: 138 times
- Price Per Unit: 533 errors
  Invalid Values:
    - nan: 179 times
    - 'ERROR': 190 times
    - 'UNKNOWN': 164 times
- Total Spent: 502 errors
  Invalid Values:
    - 'ERROR': 164 times
    - nan: 173 times
    - 'UNKNOWN': 165 times
- Payment Method: 2579 errors
  Invalid Values:
    - nan: 2579 times
- Location: 3265 errors
  Invalid Values:
    - nan: 3265 times
- Transaction Date: 460 errors
  Invalid Values:
    - 'ERROR': 142 times
    - nan: 159 times
    - 'UNKNOWN': 159 times


###3.2 Range Errors

In [9]:
# PARAMETERS

range_rules = {
    "Quantity": {"min": 1, "max": 100},
    "Price Per Unit": {"min": 0.01, "max": 5}, # According to the menu in the kaggle dataset description
    "Total Spent": {"min": 0, "max": 1000}
}

In [10]:
# CODE

range_errors = {}

for col, bounds in range_rules.items():
    if col in ds.columns:
        out_of_range_values = []
        non_numeric_values = []
        nan_count = 0

        ds[col] = pd.to_numeric(ds[col], errors='coerce')

        # count NaN values separately
        nan_count = ds[col].isna().sum()

        # store non-numeric values separately (excluding NaN)
        invalid_entries = ds.loc[ds[col].isna(), col]
        invalid_entries = invalid_entries.dropna()  # Remove NaN to avoid duplicates
        if not invalid_entries.empty:
            non_numeric_values.extend(invalid_entries.tolist())

        if "min" in bounds:
            min_violations = ds[(ds[col] < bounds["min"]) & ds[col].notna()][col].tolist()
            out_of_range_values.extend(min_violations)

        if "max" in bounds:
            max_violations = ds[(ds[col] > bounds["max"]) & ds[col].notna()][col].tolist()
            out_of_range_values.extend(max_violations)

        if out_of_range_values or non_numeric_values or nan_count > 0:
            range_errors[col] = {
                "Total Errors": len(out_of_range_values) + len(non_numeric_values) + nan_count,
                "Out-of-Range Values": dict(Counter(out_of_range_values)),
                "Invalid (Non-Numeric) Values": dict(Counter(non_numeric_values)),
                "NaN Count": nan_count  # store NaN separately
            }
    else:
        range_errors[col] = {
            "Total Errors": "Column Missing",
            "Out-of-Range Values": {},
            "Invalid (Non-Numeric) Values": {},
            "NaN Count": 0
        }

# results
if range_errors:
    print("Range Errors Found:")
    for col, details in range_errors.items():
        print(f"- {col}: {details['Total Errors']} errors")

        if details["NaN Count"] > 0:
            print(f"  NaN Values: {details['NaN Count']} times")

        if details["Invalid (Non-Numeric) Values"]:
            print("  Non-Numeric Values:")
            for value, count in details["Invalid (Non-Numeric) Values"].items():
                print(f"    - {repr(value)}: {count} times")

        if details["Out-of-Range Values"]:
            print("  Out-of-Range Values:")
            for value, count in details["Out-of-Range Values"].items():
                print(f"    - {value}: {count} times")
else:
    print("No range errors found.")


Range Errors Found:
- Quantity: 479 errors
  NaN Values: 479 times
- Price Per Unit: 533 errors
  NaN Values: 533 times
- Total Spent: 502 errors
  NaN Values: 502 times


###3.3 Format Errors

In [11]:
# PARAMETERS

# using regex
format_rules = {
    "Transaction ID": r"^TXN_\d{7}$",  # TXN_ followed by 7 digits
    "Transaction Date": r"^\d{4}-\d{2}-\d{2}$"  # YYYY-MM-DD
}


In [12]:
# CODE

import re

format_errors = {}

for col, pattern in format_rules.items():
    if col in ds.columns:
        invalid_values = []

        # values are treated as strings before checking format
        invalid_entries = ds[~ds[col].astype(str).str.match(pattern, na=False)][col]
        if not invalid_entries.empty:
            invalid_values.extend(invalid_entries.tolist())

        if invalid_values:
            error_counts = dict(Counter(invalid_values))
            total_errors = sum(error_counts.values())
            format_errors[col] = {
                "Total Errors": total_errors,
                "Invalid Format Values": error_counts
            }
    else:
        format_errors[col] = {
            "Total Errors": "Column Missing",
            "Invalid Format Values": {}
        }

# results
if format_errors:
    print("Format Errors Found:")
    for col, details in format_errors.items():
        print(f"- {col}: {details['Total Errors']} errors")

        if details["Invalid Format Values"]:
            print("  Invalid Format Values:")
            for value, count in details["Invalid Format Values"].items():
                print(f"    - {repr(value)}: {count} times")
else:
    print("No format errors found.")


Format Errors Found:
- Transaction Date: 460 errors
  Invalid Format Values:
    - 'ERROR': 142 times
    - nan: 159 times
    - 'UNKNOWN': 159 times


###3.4 Consistency Errors

In [13]:
# PARAMETERS

consistency_rule = {
    "Total Spent": "Price Per Unit * Quantity"
}


In [14]:
# CODE

consistency_errors = {}

# convert numerical cols
ds["Quantity"] = pd.to_numeric(ds["Quantity"], errors='coerce')
ds["Price Per Unit"] = pd.to_numeric(ds["Price Per Unit"], errors='coerce')
ds["Total Spent"] = pd.to_numeric(ds["Total Spent"], errors='coerce')

expected_total_spent = ds["Price Per Unit"] * ds["Quantity"]

# flag when total spent is not equal OR when value is missing
inconsistent_rows = ds[
    (ds["Total Spent"].isna()) | (ds["Price Per Unit"].isna()) | (ds["Quantity"].isna()) |
    (np.abs(ds["Total Spent"] - expected_total_spent) > 0.00)
]

total_inconsistencies = len(inconsistent_rows)
invalid_price_count = 0
invalid_quantity_count = 0
invalid_total_spent_count = 0

# cause of inconsistency
for _, row in inconsistent_rows.iterrows():
    is_invalid_total = pd.isna(row["Total Spent"]) or np.abs(row["Total Spent"] - (row["Price Per Unit"] * row["Quantity"])) > 0.01
    is_invalid_price = pd.isna(row["Price Per Unit"]) or row["Price Per Unit"] <= 0
    is_invalid_quantity = pd.isna(row["Quantity"]) or row["Quantity"] <= 0

    if is_invalid_price:
        invalid_price_count += 1
    if is_invalid_quantity:
        invalid_quantity_count += 1
    if is_invalid_total:
        invalid_total_spent_count += 1

consistency_errors["Total Spent"] = {
    "Total Inconsistencies": total_inconsistencies,
    "Due to Invalid Price Per Unit": invalid_price_count,
    "Due to Invalid Quantity": invalid_quantity_count,
    "Due to Invalid Total Spent": invalid_total_spent_count
}

# results
if consistency_errors["Total Spent"]["Total Inconsistencies"] > 0:
    print("Consistency Errors Found:")
    print(f"- Total Spent: {consistency_errors['Total Spent']['Total Inconsistencies']} inconsistencies")
    print(f"  - Due to Invalid Price Per Unit: {consistency_errors['Total Spent']['Due to Invalid Price Per Unit']}")
    print(f"  - Due to Invalid Quantity: {consistency_errors['Total Spent']['Due to Invalid Quantity']}")
    print(f"  - Due to Invalid Total Spent: {consistency_errors['Total Spent']['Due to Invalid Total Spent']}")
else:
    print("No consistency errors found.")

Consistency Errors Found:
- Total Spent: 1456 inconsistencies
  - Due to Invalid Price Per Unit: 533
  - Due to Invalid Quantity: 479
  - Due to Invalid Total Spent: 502


###3.5 Uniqueness Errors

In [15]:
# PARAMETERS

# Columns that must be unique
uniqueness_rule = [
    "Transaction ID"
]

In [16]:
# CODE

uniqueness_errors = {}

for col in uniqueness_rule:
    if col in ds.columns:
        duplicate_values = ds[col][ds[col].duplicated(keep=False)]

        if not duplicate_values.empty:
            error_counts = dict(Counter(duplicate_values))
            total_errors = len(duplicate_values)
            uniqueness_errors[col] = {
                "Total Duplicates": total_errors,
                "Duplicate Values": error_counts
            }
    else:
        uniqueness_errors[col] = {
            "Total Duplicates": "Column Missing",
            "Duplicate Values": {}
        }

# results
if uniqueness_errors:
    print("Uniqueness Errors Found:")
    for col, details in uniqueness_errors.items():
        print(f"- {col}: {details['Total Duplicates']} duplicate entries")
        if details["Duplicate Values"]:
            print("  Duplicate Values:")
            for value, count in details["Duplicate Values"].items():
                print(f"    - {repr(value)}: {count} times")
else:
    print("No uniqueness errors found.")


No uniqueness errors found.


###3.6 Presence Errors

In [17]:
# PARAMETERS

# Columns that must not have missing values
presence_rule = [
    "Transaction ID", "Item", "Quantity", "Price Per Unit", "Total Spent", "Payment Method", "Location", "Transaction Date"
    ]


In [18]:
# CODE

presence_errors = {}

for col in presence_rule:
    if col in ds.columns:
        missing_count = ds[col].isna().sum()  # Count NaN values
        if missing_count > 0:
            presence_errors[col] = missing_count
    else:
        presence_errors[col] = "Column Missing"

# Result
if presence_errors:
    print("Presence Errors Found:")
    for col, missing_count in presence_errors.items():
        print(f"- {col}: {missing_count} missing values")
else:
    print("No presence errors found.")


Presence Errors Found:
- Item: 333 missing values
- Quantity: 479 missing values
- Price Per Unit: 533 missing values
- Total Spent: 502 missing values
- Payment Method: 2579 missing values
- Location: 3265 missing values
- Transaction Date: 159 missing values


###3.7 Length Errors

In [19]:
# PARAMETERS

length_rule = {
    "Transaction ID": 11,  # TXN_1234567
    "Transaction Date": 10  # YYYY-MM-DD
}


In [20]:
# CODE

length_errors = {}

for col, expected_length in length_rule.items():
    if col in ds.columns:
        # Convert to string before checking length
        invalid_length_values = ds[ds[col].astype(str).str.len() != expected_length][col]

        if not invalid_length_values.empty:
            length_errors[col] = {
                "Total Errors": len(invalid_length_values),
                "Invalid Values": dict(Counter(invalid_length_values.tolist()))
            }
    else:
        length_errors[col] = {
            "Total Errors": "Column Missing",
            "Invalid Values": {}
        }

# results
if length_errors:
    print("Length Errors Found:")
    for col, details in length_errors.items():
        print(f"- {col}: {details['Total Errors']} incorrect length values")
        if details["Invalid Values"]:
            print("  Invalid Values:")
            for value, count in details["Invalid Values"].items():
                print(f"    - {repr(value)}: {count} times")
else:
    print("No length errors found.")


Length Errors Found:
- Transaction Date: 460 incorrect length values
  Invalid Values:
    - 'ERROR': 142 times
    - nan: 159 times
    - 'UNKNOWN': 159 times


###3.8 Look-Up Errors

In [21]:
# PARAMETERS

lookup_rule = {
    "Item": ["Coffee", "Tea", "Sandwich", "Salad", "Cake", "Cookie", "Smoothie", "Juice"],
    "Price Per Unit": [2.0, 1.5, 4.0, 5.0, 3.0, 1.0],
    "Payment Method": ["Cash", "Credit Card", "Digital Wallet"],
    "Location": ["In-Store", "Takeaway"]
}


In [22]:
# CODE

lookup_errors = {}

for col, valid_values in lookup_rule.items():
    if col in ds.columns:
        invalid_values = ds[~ds[col].isin(valid_values)][col]

        if not invalid_values.empty:
            # count NaN values separately
            nan_count = invalid_values.isna().sum()

            # Remove NaN before counting other invalid values
            filtered_invalid_values = invalid_values.dropna()
            error_counts = dict(Counter(filtered_invalid_values.tolist()))

            # Add NaN count if applicable
            if nan_count > 0:
                error_counts["NaN"] = nan_count

            lookup_errors[col] = {
                "Total Errors": len(invalid_values),
                "Invalid Values": error_counts
            }
    else:
        lookup_errors[col] = {
            "Total Errors": "Column Missing",
            "Invalid Values": {}
        }

# results
if lookup_errors:
    print("Look-up Errors Found:")
    for col, details in lookup_errors.items():
        print(f"- {col}: {details['Total Errors']} invalid values")
        if details["Invalid Values"]:
            print("  Invalid Values:")
            for value, count in details["Invalid Values"].items():
                print(f"    - {repr(value)}: {count} times")
else:
    print("No look-up errors found.")



Look-up Errors Found:
- Item: 969 invalid values
  Invalid Values:
    - 'UNKNOWN': 344 times
    - 'ERROR': 292 times
    - 'NaN': 333 times
- Price Per Unit: 533 invalid values
  Invalid Values:
    - 'NaN': 533 times
- Payment Method: 3178 invalid values
  Invalid Values:
    - 'UNKNOWN': 293 times
    - 'ERROR': 306 times
    - 'NaN': 2579 times
- Location: 6978 invalid values
  Invalid Values:
    - 'In-store': 3017 times
    - 'UNKNOWN': 338 times
    - 'ERROR': 358 times
    - 'NaN': 3265 times


###3.9 Exact Duplicate Errors

In [23]:
# PARAMETERS

# Exact duplicate rule: The entire row should be unique
exact_duplicate_rule = "All columns must be unique as a full row"


In [24]:
# CODE

exact_duplicates = ds[ds.duplicated(keep=False)]

total_exact_duplicates = len(exact_duplicates)

# results
if total_exact_duplicates > 0:
    print(f"Exact Duplicate Errors Found: {total_exact_duplicates} duplicate rows")

    # Show a few examples
    print("Sample Duplicate Rows:")
    print(exact_duplicates.head(5))  # Show first 5 duplicates
else:
    print("No exact duplicate rows found.")


No exact duplicate rows found.


###3.10 Near Duplicate Errors

In [25]:
# PARAMETERS

near_duplicate_rule = ["Transaction ID", "Item", "Quantity", "Price Per Unit", "Total Spent", "Payment Method", "Location", "Transaction Date"]

In [26]:
!pip install fuzzywuzzy

In [27]:
# CODE

from collections import defaultdict
from fuzzywuzzy import fuzz

key_columns = ["Transaction ID", "Item", "Quantity", "Price Per Unit", "Total Spent", "Payment Method", "Location", "Transaction Date"]
grouped_rows = defaultdict(list)

for idx, row in ds.iterrows():
    key = (row["Item"], row["Transaction Date"])
    grouped_rows[key].append(idx)

near_duplicates = []

# similarity thresholds
TEXT_SIMILARITY_THRESHOLD = 90  # fuzzy matching
NUMERIC_TOLERANCE = 0.1

for key, indices in grouped_rows.items():
    for i in range(len(indices)):
        for j in range(i + 1, len(indices)):
            row1, row2 = ds.iloc[indices[i]], ds.iloc[indices[j]]
            differences = 0

            # check text similarity
            for col in ["Transaction ID", "Item", "Payment Method", "Location", "Transaction Date"]:
                if fuzz.ratio(str(row1[col]), str(row2[col])) < TEXT_SIMILARITY_THRESHOLD:
                    differences += 1

            # check numeric values within tolerance
            for col in ["Quantity", "Price Per Unit", "Total Spent"]:
                if pd.notna(row1[col]) and pd.notna(row2[col]):  # Ignore NaN values
                    if abs(row1[col] - row2[col]) > NUMERIC_TOLERANCE:
                        differences += 1

            # near duplicate = 1 or 2 fields are different
            if 1 <= differences <= 2:
                near_duplicates.append((indices[i], indices[j]))

# results
if near_duplicates:
    print(f"Near Duplicate Errors Found: {len(near_duplicates)} near-duplicate row pairs")

    # show some examples
    print("Sample Near-Duplicate Rows:")
    for i, j in near_duplicates[:5]:
        print("\nPair:")
        print(ds.iloc[i])
        print(ds.iloc[j])
else:
    print("No near duplicate rows found.")




Near Duplicate Errors Found: 1686 near-duplicate row pairs
Sample Near-Duplicate Rows:

Pair:
Transaction ID      TXN_4271903
Item                     Cookie
Quantity                    4.0
Price Per Unit              1.0
Total Spent                 NaN
Payment Method      Credit Card
Location               In-store
Transaction Date     2023-07-19
Name: 2, dtype: object
Transaction ID      TXN_8477945
Item                     Cookie
Quantity                    4.0
Price Per Unit              1.0
Total Spent                 4.0
Payment Method              NaN
Location               In-store
Transaction Date     2023-07-19
Name: 6995, dtype: object

Pair:
Transaction ID      TXN_2602893
Item                   Smoothie
Quantity                    5.0
Price Per Unit              4.0
Total Spent                20.0
Payment Method      Credit Card
Location                    NaN
Transaction Date     2023-03-31
Name: 5, dtype: object
Transaction ID      TXN_1967565
Item                   Smoo

## References
* https://stackoverflow.com/questions/53622279/fuzzy-match-rows-in-single-dataframe-to-find-duplicates-in-pandas-and-python
* https://www.youtube.com/watch?v=lCFEzRaqoJA&ab_channel=KeithLyons